In [1]:
import pickle
import numpy as np
from keras import backend as K

participantCount = 32

rawData = [{}]*participantCount

for i in range(participantCount):
    rawData[i] = pickle.load(open(f'../data/s{i+1:02}.dat', 'rb'), encoding="latin1")

labels = np.array(list(map(lambda participant : participant['labels'], rawData)))
data = np.array(list(map(lambda participant : participant['data'], rawData)))

def get_y(emotion):
    return (labels.reshape(-1, 4)[:,emotion] >= 5).astype(int)

def get_peripheral_x():
    return data[:, :, 32:, :].reshape(-1, 8, 8064)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [2]:
from matplotlib import pyplot
from sklearn.model_selection import KFold
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Y = get_y(0)
X = np.moveaxis(get_peripheral_x(), 1, 2)

for i in range(4):
    print("EMOTION NUMBER", i)
    Y = get_y(i)
    kfold = KFold(n_splits=5, shuffle=False)
    cvscores = []
    fscores = []
    for train, test in kfold.split(X, Y):
        model = Sequential()
        model.add(LSTM(80, input_dim=8))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc", f1_m, precision_m, recall_m])

        model.fit(X[train], Y[train], validation_data=(X[test], Y[test]), epochs=100, batch_size=128, verbose=2, shuffle=False)

        scores = model.evaluate(X[test], Y[test], verbose=0)
        print("%s: %.2f%%, %s: %.2f%%" % (model.metrics_names[1], scores[1]*100, model.metrics_names[2], scores[2]*100))
        cvscores.append(scores[1] * 100)
        fscores.append(scores[2] * 100)

    print("%.2f%% (+/- %.2f%%), %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores), np.mean(fscores), np.std(fscores)))    
    print("EMOTION", i)

EMOTION NUMBER 0
Epoch 1/100
8/8 - 3s - loss: 0.7577 - acc: 0.4863 - f1_m: 0.3854 - precision_m: 0.5419 - recall_m: 0.3375 - val_loss: 0.7638 - val_acc: 0.4688 - val_f1_m: 0.4429 - val_precision_m: 0.5807 - val_recall_m: 0.3605
Epoch 2/100
8/8 - 3s - loss: 0.7162 - acc: 0.5264 - f1_m: 0.4723 - precision_m: 0.5853 - recall_m: 0.4237 - val_loss: 0.7524 - val_acc: 0.5000 - val_f1_m: 0.5270 - val_precision_m: 0.5965 - val_recall_m: 0.4731
Epoch 3/100
8/8 - 3s - loss: 0.6942 - acc: 0.5381 - f1_m: 0.5154 - precision_m: 0.5865 - recall_m: 0.4786 - val_loss: 0.7458 - val_acc: 0.4805 - val_f1_m: 0.5231 - val_precision_m: 0.5695 - val_recall_m: 0.4879
Epoch 4/100
8/8 - 3s - loss: 0.6808 - acc: 0.5596 - f1_m: 0.5609 - precision_m: 0.6022 - recall_m: 0.5386 - val_loss: 0.7379 - val_acc: 0.4844 - val_f1_m: 0.5596 - val_precision_m: 0.5645 - val_recall_m: 0.5607
Epoch 5/100
8/8 - 3s - loss: 0.6725 - acc: 0.5645 - f1_m: 0.5782 - precision_m: 0.5993 - recall_m: 0.5732 - val_loss: 0.7344 - val_acc: 0.5

Epoch 40/100
8/8 - 3s - loss: 0.5822 - acc: 0.6934 - f1_m: 0.7384 - precision_m: 0.6929 - recall_m: 0.7929 - val_loss: 0.7016 - val_acc: 0.5469 - val_f1_m: 0.6176 - val_precision_m: 0.6138 - val_recall_m: 0.6216
Epoch 41/100
8/8 - 3s - loss: 0.5797 - acc: 0.6885 - f1_m: 0.7330 - precision_m: 0.6905 - recall_m: 0.7842 - val_loss: 0.6989 - val_acc: 0.5391 - val_f1_m: 0.6127 - val_precision_m: 0.6047 - val_recall_m: 0.6208
Epoch 42/100
8/8 - 3s - loss: 0.5813 - acc: 0.6875 - f1_m: 0.7315 - precision_m: 0.6904 - recall_m: 0.7801 - val_loss: 0.6971 - val_acc: 0.5234 - val_f1_m: 0.5972 - val_precision_m: 0.5933 - val_recall_m: 0.6012
Epoch 43/100
8/8 - 3s - loss: 0.5772 - acc: 0.6943 - f1_m: 0.7366 - precision_m: 0.6966 - recall_m: 0.7839 - val_loss: 0.6985 - val_acc: 0.5234 - val_f1_m: 0.6002 - val_precision_m: 0.5928 - val_recall_m: 0.6083
Epoch 44/100
8/8 - 3s - loss: 0.5746 - acc: 0.6963 - f1_m: 0.7390 - precision_m: 0.6992 - recall_m: 0.7866 - val_loss: 0.6928 - val_acc: 0.5312 - val_f1

Epoch 79/100
8/8 - 3s - loss: 0.5360 - acc: 0.7236 - f1_m: 0.7609 - precision_m: 0.7228 - recall_m: 0.8070 - val_loss: 0.7206 - val_acc: 0.5195 - val_f1_m: 0.5777 - val_precision_m: 0.6001 - val_recall_m: 0.5583
Epoch 80/100
8/8 - 3s - loss: 0.5369 - acc: 0.7246 - f1_m: 0.7610 - precision_m: 0.7257 - recall_m: 0.8040 - val_loss: 0.7161 - val_acc: 0.5391 - val_f1_m: 0.5941 - val_precision_m: 0.6190 - val_recall_m: 0.5716
Epoch 81/100
8/8 - 3s - loss: 0.5360 - acc: 0.7266 - f1_m: 0.7621 - precision_m: 0.7269 - recall_m: 0.8050 - val_loss: 0.7173 - val_acc: 0.5391 - val_f1_m: 0.5972 - val_precision_m: 0.6175 - val_recall_m: 0.5786
Epoch 82/100
8/8 - 3s - loss: 0.5356 - acc: 0.7285 - f1_m: 0.7628 - precision_m: 0.7292 - recall_m: 0.8029 - val_loss: 0.7174 - val_acc: 0.5391 - val_f1_m: 0.5948 - val_precision_m: 0.6189 - val_recall_m: 0.5732
Epoch 83/100
8/8 - 3s - loss: 0.5329 - acc: 0.7334 - f1_m: 0.7673 - precision_m: 0.7319 - recall_m: 0.8093 - val_loss: 0.7183 - val_acc: 0.5234 - val_f1

Epoch 18/100
8/8 - 3s - loss: 0.5864 - acc: 0.6904 - f1_m: 0.7505 - precision_m: 0.6971 - recall_m: 0.8256 - val_loss: 0.7101 - val_acc: 0.5352 - val_f1_m: 0.5840 - val_precision_m: 0.5691 - val_recall_m: 0.6001
Epoch 19/100
8/8 - 3s - loss: 0.5823 - acc: 0.7012 - f1_m: 0.7601 - precision_m: 0.7046 - recall_m: 0.8376 - val_loss: 0.7094 - val_acc: 0.5391 - val_f1_m: 0.5907 - val_precision_m: 0.5704 - val_recall_m: 0.6127
Epoch 20/100
8/8 - 3s - loss: 0.5794 - acc: 0.7051 - f1_m: 0.7639 - precision_m: 0.7057 - recall_m: 0.8448 - val_loss: 0.7056 - val_acc: 0.5234 - val_f1_m: 0.5766 - val_precision_m: 0.5571 - val_recall_m: 0.5992
Epoch 21/100
8/8 - 3s - loss: 0.5761 - acc: 0.7070 - f1_m: 0.7658 - precision_m: 0.7077 - recall_m: 0.8476 - val_loss: 0.7069 - val_acc: 0.5234 - val_f1_m: 0.5800 - val_precision_m: 0.5563 - val_recall_m: 0.6069
Epoch 22/100
8/8 - 3s - loss: 0.5753 - acc: 0.7041 - f1_m: 0.7618 - precision_m: 0.7068 - recall_m: 0.8390 - val_loss: 0.7055 - val_acc: 0.5078 - val_f1

Epoch 57/100
8/8 - 3s - loss: 0.5242 - acc: 0.7432 - f1_m: 0.7849 - precision_m: 0.7457 - recall_m: 0.8346 - val_loss: 0.7424 - val_acc: 0.5195 - val_f1_m: 0.5778 - val_precision_m: 0.5497 - val_recall_m: 0.6089
Epoch 58/100
8/8 - 3s - loss: 0.5243 - acc: 0.7432 - f1_m: 0.7842 - precision_m: 0.7474 - recall_m: 0.8323 - val_loss: 0.7398 - val_acc: 0.5273 - val_f1_m: 0.5819 - val_precision_m: 0.5573 - val_recall_m: 0.6089
Epoch 59/100
8/8 - 3s - loss: 0.5222 - acc: 0.7451 - f1_m: 0.7844 - precision_m: 0.7500 - recall_m: 0.8279 - val_loss: 0.7429 - val_acc: 0.5234 - val_f1_m: 0.5669 - val_precision_m: 0.5564 - val_recall_m: 0.5791
Epoch 60/100
8/8 - 3s - loss: 0.5203 - acc: 0.7461 - f1_m: 0.7818 - precision_m: 0.7588 - recall_m: 0.8128 - val_loss: 0.7456 - val_acc: 0.5117 - val_f1_m: 0.5598 - val_precision_m: 0.5478 - val_recall_m: 0.5733
Epoch 61/100
8/8 - 3s - loss: 0.5186 - acc: 0.7490 - f1_m: 0.7799 - precision_m: 0.7659 - recall_m: 0.8013 - val_loss: 0.7507 - val_acc: 0.5156 - val_f1

...

Epoch 34/100
8/8 - 3s - loss: 0.5277 - acc: 0.7520 - f1_m: 0.8436 - precision_m: 0.7475 - recall_m: 0.9813 - val_loss: 0.7414 - val_acc: 0.5586 - val_f1_m: 0.6971 - val_precision_m: 0.6061 - val_recall_m: 0.8204
Epoch 35/100
8/8 - 3s - loss: 0.5257 - acc: 0.7539 - f1_m: 0.8450 - precision_m: 0.7485 - recall_m: 0.9831 - val_loss: 0.7436 - val_acc: 0.5547 - val_f1_m: 0.6907 - val_precision_m: 0.6042 - val_recall_m: 0.8072
Epoch 36/100
8/8 - 3s - loss: 0.5273 - acc: 0.7520 - f1_m: 0.8442 - precision_m: 0.7465 - recall_m: 0.9846 - val_loss: 0.7425 - val_acc: 0.5508 - val_f1_m: 0.6888 - val_precision_m: 0.6014 - val_recall_m: 0.8072
Epoch 37/100
8/8 - 3s - loss: 0.5275 - acc: 0.7539 - f1_m: 0.8452 - precision_m: 0.7483 - recall_m: 0.9846 - val_loss: 0.7433 - val_acc: 0.5586 - val_f1_m: 0.6936 - val_precision_m: 0.6079 - val_recall_m: 0.8080
Epoch 38/100
8/8 - 3s - loss: 0.5233 - acc: 0.7568 - f1_m: 0.8463 - precision_m: 0.7525 - recall_m: 0.9800 - val_loss: 0.7449 - val_acc: 0.5391 - val_f1

Epoch 73/100
8/8 - 3s - loss: 0.4978 - acc: 0.7754 - f1_m: 0.8566 - precision_m: 0.7758 - recall_m: 0.9692 - val_loss: 0.7696 - val_acc: 0.5234 - val_f1_m: 0.6577 - val_precision_m: 0.5927 - val_recall_m: 0.7402
Epoch 74/100
8/8 - 3s - loss: 0.4971 - acc: 0.7773 - f1_m: 0.8578 - precision_m: 0.7759 - recall_m: 0.9719 - val_loss: 0.7734 - val_acc: 0.5430 - val_f1_m: 0.6741 - val_precision_m: 0.6063 - val_recall_m: 0.7599
Epoch 75/100
8/8 - 3s - loss: 0.4953 - acc: 0.7822 - f1_m: 0.8609 - precision_m: 0.7774 - recall_m: 0.9774 - val_loss: 0.7751 - val_acc: 0.5391 - val_f1_m: 0.6699 - val_precision_m: 0.6039 - val_recall_m: 0.7534
Epoch 76/100
8/8 - 3s - loss: 0.4929 - acc: 0.7852 - f1_m: 0.8633 - precision_m: 0.7819 - recall_m: 0.9777 - val_loss: 0.7725 - val_acc: 0.5469 - val_f1_m: 0.6712 - val_precision_m: 0.6108 - val_recall_m: 0.7468
Epoch 77/100
8/8 - 3s - loss: 0.4931 - acc: 0.7832 - f1_m: 0.8618 - precision_m: 0.7801 - recall_m: 0.9761 - val_loss: 0.7702 - val_acc: 0.5391 - val_f1

Epoch 12/100
8/8 - 3s - loss: 0.5915 - acc: 0.7012 - f1_m: 0.8082 - precision_m: 0.7122 - recall_m: 0.9422 - val_loss: 0.6901 - val_acc: 0.5547 - val_f1_m: 0.6125 - val_precision_m: 0.6814 - val_recall_m: 0.5653
Epoch 13/100
8/8 - 3s - loss: 0.5890 - acc: 0.7041 - f1_m: 0.8115 - precision_m: 0.7120 - recall_m: 0.9513 - val_loss: 0.6887 - val_acc: 0.5508 - val_f1_m: 0.6101 - val_precision_m: 0.6784 - val_recall_m: 0.5640
Epoch 14/100
8/8 - 3s - loss: 0.5890 - acc: 0.7031 - f1_m: 0.8108 - precision_m: 0.7105 - recall_m: 0.9511 - val_loss: 0.6856 - val_acc: 0.5586 - val_f1_m: 0.6195 - val_precision_m: 0.6831 - val_recall_m: 0.5765
Epoch 15/100
8/8 - 3s - loss: 0.5867 - acc: 0.7002 - f1_m: 0.8086 - precision_m: 0.7102 - recall_m: 0.9458 - val_loss: 0.6883 - val_acc: 0.5664 - val_f1_m: 0.6288 - val_precision_m: 0.6858 - val_recall_m: 0.5891
Epoch 16/100
8/8 - 3s - loss: 0.5830 - acc: 0.7051 - f1_m: 0.8116 - precision_m: 0.7153 - recall_m: 0.9461 - val_loss: 0.6894 - val_acc: 0.5508 - val_f1

Epoch 51/100
8/8 - 3s - loss: 0.5454 - acc: 0.7324 - f1_m: 0.8303 - precision_m: 0.7311 - recall_m: 0.9678 - val_loss: 0.7290 - val_acc: 0.5430 - val_f1_m: 0.6185 - val_precision_m: 0.6545 - val_recall_m: 0.5934
Epoch 52/100
8/8 - 3s - loss: 0.5427 - acc: 0.7354 - f1_m: 0.8321 - precision_m: 0.7338 - recall_m: 0.9683 - val_loss: 0.7286 - val_acc: 0.5352 - val_f1_m: 0.6118 - val_precision_m: 0.6480 - val_recall_m: 0.5865
Epoch 53/100
8/8 - 3s - loss: 0.5415 - acc: 0.7354 - f1_m: 0.8318 - precision_m: 0.7350 - recall_m: 0.9659 - val_loss: 0.7298 - val_acc: 0.5352 - val_f1_m: 0.6113 - val_precision_m: 0.6472 - val_recall_m: 0.5851
Epoch 54/100
8/8 - 3s - loss: 0.5425 - acc: 0.7422 - f1_m: 0.8363 - precision_m: 0.7384 - recall_m: 0.9719 - val_loss: 0.7273 - val_acc: 0.5312 - val_f1_m: 0.6170 - val_precision_m: 0.6362 - val_recall_m: 0.6046
Epoch 55/100
8/8 - 3s - loss: 0.5374 - acc: 0.7412 - f1_m: 0.8350 - precision_m: 0.7372 - recall_m: 0.9696 - val_loss: 0.7254 - val_acc: 0.5312 - val_f1

Epoch 90/100
8/8 - 3s - loss: 0.5263 - acc: 0.7402 - f1_m: 0.8340 - precision_m: 0.7403 - recall_m: 0.9627 - val_loss: 0.7519 - val_acc: 0.5781 - val_f1_m: 0.6739 - val_precision_m: 0.6560 - val_recall_m: 0.6982
Epoch 91/100
8/8 - 3s - loss: 0.5256 - acc: 0.7324 - f1_m: 0.8293 - precision_m: 0.7339 - recall_m: 0.9611 - val_loss: 0.7494 - val_acc: 0.5703 - val_f1_m: 0.6662 - val_precision_m: 0.6525 - val_recall_m: 0.6870
Epoch 92/100
8/8 - 3s - loss: 0.5250 - acc: 0.7363 - f1_m: 0.8314 - precision_m: 0.7387 - recall_m: 0.9588 - val_loss: 0.7551 - val_acc: 0.5625 - val_f1_m: 0.6622 - val_precision_m: 0.6458 - val_recall_m: 0.6870
Epoch 93/100
8/8 - 3s - loss: 0.5269 - acc: 0.7383 - f1_m: 0.8321 - precision_m: 0.7400 - recall_m: 0.9579 - val_loss: 0.7604 - val_acc: 0.5703 - val_f1_m: 0.6698 - val_precision_m: 0.6493 - val_recall_m: 0.6982
Epoch 94/100
8/8 - 3s - loss: 0.5260 - acc: 0.7383 - f1_m: 0.8326 - precision_m: 0.7373 - recall_m: 0.9636 - val_loss: 0.7558 - val_acc: 0.5742 - val_f1